# Agri Advisor api

In [1]:
import os
import openai
openai.api_key  = ''


In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [6]:
response = get_completion("write a types of agriculture")

In [7]:
print(response)

1. Subsistence agriculture: This type of agriculture is practiced by small-scale farmers who grow crops and raise livestock primarily to feed themselves and their families. The focus is on producing enough food to meet their basic needs.

2. Commercial agriculture: Commercial agriculture involves the production of crops and livestock for sale in the market. Farmers in this type of agriculture typically use modern farming techniques and equipment to maximize their yields and profits.

3. Organic agriculture: Organic agriculture is a type of farming that relies on natural methods and practices to grow crops and raise livestock. This includes avoiding the use of synthetic pesticides, fertilizers, and genetically modified organisms.

4. Intensive agriculture: Intensive agriculture involves the use of high inputs of labor, capital, and technology to maximize crop yields on a limited amount of land. This type of farming often relies on irrigation, fertilizers, and pesticides to achieve high 

In [9]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut
    )
    return response.choices[0].message["content"]

In [10]:
messages =  [
{'role':'system',
 'content':"""You are an assistant of Agro Adviser API."""},
{'role':'user',
 'content':"""types of agriculture lands"""},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

There are several types of agricultural lands based on different criteria. Here are some common types of agricultural lands:

1. Arable Land: Land that is suitable for growing crops.
2. Pasture Land: Land used for grazing livestock.
3. Rangeland: Open land for grazing that may not be suitable for crop cultivation.
4. Orchard Land: Land used for growing fruit trees.
5. Vineyard Land: Land used for growing grapevines for wine production.
6. Forest Land: Land covered with trees, which can also be used for timber production.
7. Wetland: Land that is covered with water for at least part of the year, used for rice cultivation, aquaculture, etc.
8. Desert Land: Land in arid regions where specialized farming practices are required, such as desert agriculture or desert farming.

These are just a few examples, and there can be many other types of agricultural lands based on various factors such as soil type, climate, and topography.
